**Generating Elasticsearch Top-k Results for Re-ranking**

In [1]:
!pip3 install elasticsearch

In [2]:
!pip3 install elasticsearch-dsl

In [3]:
from elasticsearch import Elasticsearch

import sys
sys.path.insert(0, '../../../BERT-FAQ/')

# import required classes and functions
from reranker import ReRanker
from evaluation import get_relevance_label_df

# import utility functions
from shared.utils import load_from_json
from shared.utils import dump_to_json
from shared.utils import make_dirs

In [4]:
# Generate list of test queries, relevance labels for ReRanker class
query_answer_pair_filepath = '../../../BERT-FAQ/data/StackFAQ/query_answer_pairs.json'
relevance_label_df = get_relevance_label_df(query_answer_pair_filepath)
test_queries = relevance_label_df[relevance_label_df['query_type'] == 'user_query'].question.unique()

In [5]:
# check first 5 occurences for test_queries
test_queries[:5]

array(['How to make font strikethrough on github.',
       'Is it possible to get  strikethrough letter formatting on github markdown.',
       'Making the text on github crossed out.',
       'Introducing stikethrough formatting on markdown for github.',
       'The <s> tag for font on github markdown doesnt work, is there an alternative?'],
      dtype=object)

In [6]:
test_queries = list(test_queries)

In [7]:
# check relevance_label_df
relevance_label_df

,id,question,answer,query_type,label
0,1,How to make font strikethrough on github.,Use ~~tildes around the words~~.,user_query,1
1,2,Is it possible to get strikethrough letter fo...,Use ~~tildes around the words~~.,user_query,1
2,3,Making the text on github crossed out.,Use ~~tildes around the words~~.,user_query,1
3,4,Introducing stikethrough formatting on markdow...,Use ~~tildes around the words~~.,user_query,1
4,5,The <s> tag for font on github markdown doesnt...,Use ~~tildes around the words~~.,user_query,1
...,...,...,...,...,...
7898,7899,How do I get an m dash in google docs?,"On Mac, just hold fn + option + -",user_query,1
7899,7900,Do google docs support the longer style dash (...,"On Mac, just hold fn + option + -",user_query,1
7900,7901,I desperately need the em dash in google docs ...,"On Mac, just hold fn + option + -",user_query,1
7901,7902,"I require the longer (em) dash in google docs,...","On Mac, just hold fn + option + -",user_query,1


In [8]:
# Define instance of ReRanker class
r = ReRanker(
    bert_model_path='', 
    test_queries=test_queries, relevance_label_df=relevance_label_df
)

In [9]:
# create output path to save Elasticsearch top-k results
output_path = "../../../BERT-FAQ/data/StackFAQ/rank_results/unsupervised"
make_dirs(output_path)

In [10]:
# Select top-k value
top_k = 100

In [11]:
# Get top-k Elasticsearch results 

es = Elasticsearch([{'host':'localhost','port':9200}], http_auth=('elastic', 'elastic')) 

es_query_by_question = r.get_es_topk_results(es=es, index='stackfaq', query_by=['question'], top_k=top_k)
es_query_by_answer = r.get_es_topk_results(es=es, index='stackfaq', query_by=['answer'], top_k=top_k)
es_query_by_question_answer = r.get_es_topk_results(es=es, index='stackfaq', query_by=['question', 'answer'], top_k=top_k)
es_query_by_question_answer_concat = r.get_es_topk_results(es=es, index='stackfaq', query_by=['question_answer'], top_k=top_k)

2021-06-07 14:45:27 - Generating ES top-k results ...
  0%|          | 0/1249 [00:00<?, ?it/s]2021-06-07 14:45:27 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.061s]
2021-06-07 14:45:27 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.052s]
  0%|          | 2/1249 [00:00<01:17, 16.15it/s]2021-06-07 14:45:27 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.021s]
2021-06-07 14:45:27 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.020s]
2021-06-07 14:45:27 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.020s]
2021-06-07 14:45:27 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.026s]
  0%|          | 6/1249 [00:00<01:03, 19.61it/s]2021-06-07 14:45:27 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.028s]
2021-06-07 14:45:27 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.018s]
2021-06-07 14:45:27 - POST http://localhost:9200/stackfaq/

  6%|▌         | 75/1249 [00:02<00:40, 28.67it/s]2021-06-07 14:45:29 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.033s]
2021-06-07 14:45:29 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.037s]
2021-06-07 14:45:29 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.033s]
  6%|▌         | 78/1249 [00:02<00:42, 27.80it/s]2021-06-07 14:45:29 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.031s]
2021-06-07 14:45:29 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.049s]
2021-06-07 14:45:29 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.044s]
  6%|▋         | 81/1249 [00:02<00:48, 24.25it/s]2021-06-07 14:45:30 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.030s]
2021-06-07 14:45:30 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.032s]
2021-06-07 14:45:30 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.037s]
  7%|▋ 

 12%|█▏        | 151/1249 [00:05<01:01, 17.81it/s]2021-06-07 14:45:32 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.031s]
2021-06-07 14:45:32 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.045s]
2021-06-07 14:45:32 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.068s]
 12%|█▏        | 154/1249 [00:05<01:01, 17.89it/s]2021-06-07 14:45:32 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.085s]
2021-06-07 14:45:32 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.039s]
 12%|█▏        | 156/1249 [00:05<01:05, 16.68it/s]2021-06-07 14:45:32 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.024s]
2021-06-07 14:45:32 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.031s]
2021-06-07 14:45:32 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.024s]
2021-06-07 14:45:33 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.026s]
 13%

 18%|█▊        | 225/1249 [00:08<00:41, 24.79it/s]2021-06-07 14:45:35 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.045s]
2021-06-07 14:45:35 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.035s]
2021-06-07 14:45:35 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.037s]
 18%|█▊        | 228/1249 [00:08<00:42, 24.13it/s]2021-06-07 14:45:35 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.034s]
2021-06-07 14:45:35 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.046s]
2021-06-07 14:45:36 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.030s]
 18%|█▊        | 231/1249 [00:08<00:41, 24.51it/s]2021-06-07 14:45:36 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.034s]
2021-06-07 14:45:36 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.038s]
2021-06-07 14:45:36 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.039s]
 19%

2021-06-07 14:45:39 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.036s]
2021-06-07 14:45:39 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.055s]
 24%|██▍       | 301/1249 [00:12<00:40, 23.33it/s]2021-06-07 14:45:39 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.054s]
2021-06-07 14:45:39 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.053s]
2021-06-07 14:45:39 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.041s]
 24%|██▍       | 304/1249 [00:12<00:43, 21.73it/s]2021-06-07 14:45:39 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.039s]
2021-06-07 14:45:39 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.035s]
2021-06-07 14:45:39 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.031s]
 25%|██▍       | 307/1249 [00:12<00:41, 22.85it/s]2021-06-07 14:45:39 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.037s]
2021

...

 70%|██████▉   | 874/1249 [00:29<00:13, 27.39it/s]2021-06-07 14:48:20 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.032s]
2021-06-07 14:48:20 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.031s]
2021-06-07 14:48:20 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.025s]
2021-06-07 14:48:20 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.021s]
 70%|███████   | 878/1249 [00:29<00:12, 28.94it/s]2021-06-07 14:48:20 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.041s]
2021-06-07 14:48:20 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.042s]
2021-06-07 14:48:20 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.020s]
 71%|███████   | 881/1249 [00:29<00:13, 27.83it/s]2021-06-07 14:48:20 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.023s]
2021-06-07 14:48:20 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.032s]
2021

 76%|███████▌  | 950/1249 [00:31<00:16, 18.23it/s]2021-06-07 14:48:22 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.032s]
2021-06-07 14:48:22 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.031s]
2021-06-07 14:48:22 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.031s]
 76%|███████▋  | 953/1249 [00:32<00:14, 19.98it/s]2021-06-07 14:48:23 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.030s]
2021-06-07 14:48:23 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.051s]
2021-06-07 14:48:23 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.030s]
 77%|███████▋  | 956/1249 [00:32<00:13, 20.94it/s]2021-06-07 14:48:23 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.027s]
2021-06-07 14:48:23 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.024s]
2021-06-07 14:48:23 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.032s]
2021

2021-06-07 14:48:25 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.021s]
 82%|████████▏ | 1028/1249 [00:34<00:07, 30.80it/s]2021-06-07 14:48:25 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.021s]
2021-06-07 14:48:25 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.030s]
2021-06-07 14:48:25 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.021s]
2021-06-07 14:48:25 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.017s]
 83%|████████▎ | 1032/1249 [00:34<00:06, 33.04it/s]2021-06-07 14:48:25 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.019s]
2021-06-07 14:48:25 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.023s]
2021-06-07 14:48:25 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.012s]
2021-06-07 14:48:25 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.020s]
2021-06-07 14:48:25 - POST http://localhost:9200/sta

2021-06-07 14:48:27 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.022s]
2021-06-07 14:48:27 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.015s]
2021-06-07 14:48:27 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.024s]
 89%|████████▊ | 1108/1249 [00:36<00:03, 36.19it/s]2021-06-07 14:48:27 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.008s]
2021-06-07 14:48:27 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.020s]
2021-06-07 14:48:27 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.019s]
2021-06-07 14:48:27 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.015s]
2021-06-07 14:48:27 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.018s]
 89%|████████▉ | 1113/1249 [00:36<00:03, 39.43it/s]2021-06-07 14:48:27 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.017s]
2021-06-07 14:48:27 - POST http://localhost:9200/sta

2021-06-07 14:48:29 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.028s]
2021-06-07 14:48:29 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.021s]
2021-06-07 14:48:29 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.019s]
 95%|█████████▌| 1187/1249 [00:38<00:01, 38.65it/s]2021-06-07 14:48:29 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.023s]
2021-06-07 14:48:29 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.019s]
2021-06-07 14:48:29 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.022s]
2021-06-07 14:48:29 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.019s]
2021-06-07 14:48:29 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.015s]
 95%|█████████▌| 1192/1249 [00:38<00:01, 40.30it/s]2021-06-07 14:48:29 - POST http://localhost:9200/stackfaq/_search [status:200 request:0.029s]
2021-06-07 14:48:29 - POST http://localhost:9200/sta

In [12]:
# Save Elasticsearch results to json files
dump_to_json(es_query_by_question, output_path + '/es_query_by_question.json')
dump_to_json(es_query_by_answer, output_path + '/es_query_by_answer.json')
dump_to_json(es_query_by_question_answer, output_path + '/es_query_by_question_answer.json')
dump_to_json(es_query_by_question_answer_concat, output_path + '/es_query_by_question_answer_concat.json')